# Análisis de costos de cursos en el mercado

In [1]:
import pandas as pd
import numpy as np
import os

# Rutas de archivos

In [20]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))
path_ppd = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

In [3]:
# Cargar los archivos de costos de educacion continua, costos de cursos cec y cursos de OC, OEC y CI

df_dim_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oec','dim_perfil.csv'))
df_dim_curso = pd.read_csv(os.path.join(path_dwh,'datamart_ci','dim_curso.csv'))


### Palabras clave

In [4]:
keyword = ['Python', 'programación web', 'paginas web', 'datos', 'Excel', 'Word', 'power point', 'BI', 'inteligencia de negocios', 'powerBI', 
'big data', 'programación', 'marketing', 'web', 'html', 'css',  'java', 'javascript', 'base de datos', 'redes', 'telecomunicaciones', 'Android', 'IOS', 'ARDUINO'
'Business Intelligence', 
'Desarrollo de software',
'Fibra Óptica',
'Gerencia de sistemas',
'Hackeo Ético',
'Informática Forense',
'Linux', 'WINDOWS'
'Office',
'PHP',
'Project',
'software R', 'Tableu', 'rapidminer', 
'Mysql', 'sql', 'sqlserver', 'Oracle', 'postgresql',
'Seguridad informática',
'Aulas virtuales', 'Moodle', 'software',
'Wordpress', 'joomla', 'xoops', 'drupal'
]

keyword = np.char.lower(np.array(keyword))

In [37]:
from hermetrics.levenshtein import Levenshtein
lev = Levenshtein()

In [14]:
keyword

array(['python', 'programación web', 'paginas web', 'datos', 'excel',
       'word', 'power point', 'bi', 'inteligencia de negocios', 'powerbi',
       'big data', 'programación', 'marketing', 'web', 'html', 'css',
       'java', 'javascript', 'base de datos', 'redes',
       'telecomunicaciones', 'android', 'ios',
       'arduinobusiness intelligence', 'desarrollo de software',
       'fibra óptica', 'gerencia de sistemas', 'hackeo ético',
       'informática forense', 'linux', 'windowsoffice', 'php', 'project',
       'software r', 'tableu', 'rapidminer', 'mysql', 'sql', 'sqlserver',
       'oracle', 'postgresql', 'seguridad informática', 'aulas virtuales',
       'moodle', 'software', 'wordpress', 'joomla', 'xoops', 'drupal'],
      dtype='<U28')

In [20]:
value_check = 'python'
arr = []
for i in range(len(keyword)):
    # Similaridad 
    sub_arr = []
    for j in range(len(keyword)):
        similary = lev.similarity(keyword[i],keyword[j])
        sub_arr.append(similary)
    arr.append(sub_arr)

res = pd.DataFrame(data=arr,columns=keyword,index=keyword)

In [12]:
#Filtrar las areas de Tecnologia y TICs

df_dim_curso_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[df_dim_curso_perfil['area_familia'] == 'TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN']

In [34]:
df_coec = pd.read_csv(os.path.join(path_ppd,'costos_cursos_educacion_continua.csv'))
df_coec.columns

Index(['Nombre  de los Programas', 'Modalidad', 'Precio', 'carga_horaria',
       'precio_por_hora'],
      dtype='object')

In [35]:
df_coec['Nombre  de los Programas']

0                      programming essentials in python
1                         ccna cybersecurity operations
2                        ccna1 v7: introducción a redes
3     ccna3 v7: redes empresariales, seguridad y aut...
4                         fundamentos de ciberseguridad
5                    iot fudamentals: connecting things
6                           social media data analytics
7     growth hacking: estrategias, técnicas y accion...
8               facebook e instagram para emprendedores
9     biomecánica del cuerpo humano y neurorrehabili...
10    narrativa digital: storytelling, copywriting y...
11                     el abc de ilustrador y photoshop
12                        qlik sense - business analyst
13                    \npower bi: entorno y publicación
14                    \npower bi: funciones query y dax
15               estadística para la toma de decisiones
16                    sql aplicado al análisis de datos
17    estadística aplicada a la prospección de m

In [47]:
arr = []

for i in range(len(df_coec['Nombre  de los Programas'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(df_dim_curso_perfil['curso_perfil'])):
        similary = lev.similarity(df_coec['Nombre  de los Programas'].iloc[i],df_dim_curso_perfil['curso_perfil'].iloc[j])
        sub_arr.append(similary)
    arr.append(sub_arr)


In [48]:
res = pd.DataFrame(data=arr,index=df_coec['Nombre  de los Programas'],columns=df_dim_curso_perfil['curso_perfil'])
res

curso_perfil,EXCEL BÁSICO,EXCEL INTERMEDIO,EXCEL AVANZADO,ETHICAL HACKING,AUDITORIA INFORMÁTICA,SEGURIDAD DE LA INFORMACIÓN Y SEGURIDAD INFORMÁTICA,PROGRAMA DE SEGURIDADES DE LA INFORMACIÓN E INFORMÁTICA CON ETHICAL HACKING,EXCEL INTERMEDIO,EXCEL INTERMEDIO,MANEJO Y OPERACIÓN DE UNA HOJA DE CÁLCULO,...,AUTOCAD 2D,AUTOCAD 3D,CIVIL 3D,ARCGIS BÁSICO - INTERMEDIO,DISEÑO DE SISTEMAS CONTRA INCENDIOS,EXCEL BÁSICO,EXCEL INTERMEDIO,EXCEL AVANZADO,ANÁLISIS ESTADÍSTICO CON SPSS BÁSICO,EXCEL FINANCIERO
Nombre de los Programas,,,,,,,,,,,,,,,,,,,,,
programming essentials in python,0.031250,0.031250,0.031250,0.031250,0.031250,0.058824,0.040000,0.031250,0.031250,0.073171,...,0.031250,0.031250,0.031250,0.031250,0.028571,0.031250,0.031250,0.031250,0.055556,0.031250
ccna cybersecurity operations,0.034483,0.034483,0.034483,0.034483,0.000000,0.039216,0.026667,0.034483,0.034483,0.048780,...,0.034483,0.034483,0.034483,0.034483,0.028571,0.034483,0.034483,0.034483,0.055556,0.034483
ccna1 v7: introducción a redes,0.033333,0.033333,0.033333,0.033333,0.033333,0.078431,0.053333,0.033333,0.033333,0.073171,...,0.033333,0.033333,0.033333,0.033333,0.057143,0.033333,0.033333,0.033333,0.055556,0.033333
"ccna3 v7: redes empresariales, seguridad y automatización",0.017544,0.017544,0.017544,0.017544,0.017544,0.070175,0.066667,0.017544,0.017544,0.070175,...,0.017544,0.017544,0.017544,0.052632,0.070175,0.017544,0.017544,0.017544,0.052632,0.017544
fundamentos de ciberseguridad,0.034483,0.034483,0.034483,0.034483,0.034483,0.039216,0.026667,0.034483,0.034483,0.048780,...,0.034483,0.034483,0.034483,0.034483,0.028571,0.034483,0.034483,0.034483,0.027778,0.034483
iot fudamentals: connecting things,0.029412,0.029412,0.029412,0.029412,0.029412,0.058824,0.040000,0.029412,0.029412,0.073171,...,0.029412,0.029412,0.029412,0.029412,0.000000,0.029412,0.029412,0.029412,0.027778,0.029412
social media data analytics,0.037037,0.037037,0.037037,0.037037,0.037037,0.058824,0.040000,0.037037,0.037037,0.073171,...,0.037037,0.037037,0.037037,0.037037,0.085714,0.037037,0.037037,0.037037,0.055556,0.037037
"growth hacking: estrategias, técnicas y acciones para generar tráfico a tu sitio web y redes sociales (seo, sem, analytics).",0.008065,0.008065,0.008065,0.008065,0.008065,0.048387,0.080645,0.008065,0.008065,0.056452,...,0.008065,0.008065,0.008065,0.024194,0.032258,0.008065,0.008065,0.008065,0.032258,0.008065
facebook e instagram para emprendedores,0.025641,0.025641,0.025641,0.025641,0.025641,0.058824,0.053333,0.025641,0.025641,0.048780,...,0.025641,0.025641,0.025641,0.076923,0.051282,0.025641,0.025641,0.025641,0.076923,0.025641


In [56]:
res.max().max()

0.13793103448275867

In [54]:
res['PROGRAMA DE SEGURIDADES DE LA INFORMACIÓN E INFORMÁTICA CON ETHICAL HACKING'][res['PROGRAMA DE SEGURIDADES DE LA INFORMACIÓN E INFORMÁTICA CON ETHICAL HACKING']==res['PROGRAMA DE SEGURIDADES DE LA INFORMACIÓN E INFORMÁTICA CON ETHICAL HACKING'].max()]

Nombre  de los Programas
growth hacking: estrategias, técnicas y acciones para generar tráfico a tu sitio web y redes sociales (seo, sem, analytics).    0.080645
Name: PROGRAMA DE SEGURIDADES DE LA INFORMACIÓN E INFORMÁTICA CON ETHICAL HACKING, dtype: float64

In [ ]:




# Realizar un procesado de texto en donde se contenga los nombres de los cursos por modalidad en caso de OC el resto de cursos solo se tomará un promedio general estimado de estos cursos

# Procesado de texto:
#   Aplicar a lista de las palabras clave de tecnologia
#   Validar si un curso es igual a la lista de costos de cursos
#   Buscar simetría en el nombre de los cursos en el sitio web y el nombre de los cursos de las listas de costos
#   Revisará los nombre de todos los cursos y determinará las palabras sin palabras_de_parada que se repitan más para obtener palabras clave en cuenta a cursos para buscarlas